(bash-notebook-poc)=

# Jupyter Notebook

## Step 1

In [1]:
kind create cluster --image=kindest/node:v1.26.0

Creating cluster "kind" ...
 ✓ Ensuring node image (kindest/node:v1.26.0) 🖼
 ✓ Preparing nodes 📦 7l
 ✓ Writing configuration 📜7l
 ✓ Starting control-plane 🕹️7l
 ✓ Installing CNI 🔌7l
 ✓ Installing StorageClass 💾7l
Set kubectl context to "kind-kind"
You can now use your cluster with:

kubectl cluster-info --context kind-kind

Thanks for using kind! 😊


## Step 2

In [2]:
helm install kuberay-operator kuberay/kuberay-operator --version 1.3.0

NAME: kuberay-operator
LAST DEPLOYED: Fri Feb 28 19:31:25 2025
NAMESPACE: default
STATUS: deployed
REVISION: 1
TEST SUITE: None


In [3]:
kubectl wait --for=condition=available deployment/kuberay-operator --timeout=120s

deployment.apps/kuberay-operator condition met


In [4]:
kubectl apply -f https://raw.githubusercontent.com/ray-project/kuberay/v1.3.0/ray-operator/config/samples/ray-service.mobilenet.yaml

rayservice.ray.io/rayservice-mobilenet created


In [5]:
kubectl wait --for=condition=ready rayservice/rayservice-mobilenet --timeout=500s

rayservice.ray.io/rayservice-mobilenet condition met


In [6]:
# Or use kubectl describe rayservice/rayservice-mobilenet
kubectl get rayservice/rayservice-mobilenet -o jsonpath-as-json='{.status.conditions[?(@.type=="Ready")]}'

[
    {
        "lastTransitionTime": "2025-02-28T11:35:55Z",
        "message": "Number of serve endpoints is greater than 0",
        "observedGeneration": 1,
        "reason": "NonZeroServeEndpoints",
        "status": "True",
        "type": "Ready"
    }
]


In [7]:
kubectl port-forward svc/rayservice-mobilenet-serve-svc 8000 > /dev/null &

[1] 1596457


In [8]:
python mobilenet_req.py

{"prediction":["n02892201","brass",0.0880170688033104]}


## Cleanup

In [9]:
killall kubectl
kind delete cluster

Deleting cluster "kind" ...
Deleted nodes: ["kind-control-plane"]
[1]+  Terminated              kubectl port-forward svc/rayservice-mobilenet-serve-svc 8000 > /dev/null
